In [19]:
import numpy as np
import pandas as pd
import time
from sklearn.decomposition import PCA
np.random.seed(42)

In [20]:
# function to generate synthetic stock data with broad market trend + sector trend + stock specific noise.
def generate_random_stock_data(num_stocks = 50, num_days = 1000, num_sectors = 5):

    market_trend = np.cumsum(np.random.normal(0,0.5,num_days))
    sector_trend = {i:np.cumsum(np.random.normal(0,0.3,num_days)) for i in range(num_sectors)}
    stock_specific = np.random.normal(0,0.2,(num_days,num_stocks))

    stock_sector_mapping = np.random.choice(range(num_sectors),num_stocks)

    stock_data = np.zeros((num_days,num_stocks))

    for i in range(num_stocks):
        stock_data[:,i] = 0.5*market_trend + 0.3*sector_trend[stock_sector_mapping[i]] + 0.2*stock_specific[:,i]

    stock_df = pd.DataFrame(stock_data, columns = [f'Stock_{i+1}' for i in range(num_stocks)])
    stock_df.index = pd.date_range(start = '2015-01-01', periods = num_days, freq = 'D')

    return stock_df

In [21]:
data = generate_random_stock_data()

In [22]:
# function to calculate PCA manually
def manual_pca(input_data, k):
    X = input_data.to_numpy()
    X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0, ddof=1)

    X_cov = np.cov(X_norm, rowvar=False)
    eigen_values, eigen_vectors = np.linalg.eig(X_cov)
    
    sorted_indices = np.argsort(-eigen_values)
    sorted_eigen_values = eigen_values[sorted_indices]
    sorted_eigen_vectors = eigen_vectors[:, sorted_indices]
    
    new_basis = sorted_eigen_vectors[:, :k]
    
    X_pca = X_norm @ new_basis
    X_reconstructed = X_pca @ new_basis.T
    
    explained_variance = np.sum(sorted_eigen_values[:k]) / np.sum(eigen_values)
    
    return X_pca, X_reconstructed, explained_variance, X_norm

In [23]:
# function to calculate PCA in similar output using pca from sklearn.decomposition
def sklearn_pca(input_data,k):
    X = input_data.to_numpy()
    X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0, ddof=1)

    pca = PCA(n_components = k)
    X_pca = pca.fit_transform(X_norm)
    
    X_reconstructed = X_pca @ pca.components_

    explained_variance = np.sum(pca.explained_variance_ratio_)

    return X_pca, X_reconstructed, explained_variance, X_norm

In [24]:
# benchmarking
k = 5

# Benchmark Manual PCA
start_manual = time.time()
X_pca_manual, X_reconstructed_manual, exp_var_manual, X_norm_manual = manual_pca(data, k)
time_manual = time.time() - start_manual

# Benchmark Sklearn PCA
start_sklearn = time.time()
X_pca_sklearn, X_reconstructed_sklearn, exp_var_sklearn, X_norm_sklearn = sklearn_pca(data, k)
time_sklearn = time.time() - start_sklearn

# Reconstruction error
error_manual = np.linalg.norm(X_norm_manual - X_reconstructed_manual, ord='fro')
error_sklearn = np.linalg.norm(X_norm_sklearn - X_reconstructed_sklearn, ord='fro')

In [25]:
print("\nExecution Times:")
print(f"Manual PCA time: {time_manual:.6f} seconds")
print(f"Sklearn PCA time: {time_sklearn:.6f} seconds")


Execution Times:
Manual PCA time: 0.005171 seconds
Sklearn PCA time: 0.002529 seconds


In [26]:
print("\nReconstruction Errors (Frobenius norm):")
print(f"Manual PCA reconstruction error: {error_manual:.6f}")
print(f"Sklearn PCA reconstruction error: {error_sklearn:.6f}")


Reconstruction Errors (Frobenius norm):
Manual PCA reconstruction error: 3.618004
Sklearn PCA reconstruction error: 3.618004
